In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 61.2 MB/s 
     |████████████████████████████████| 596 kB 55.9 MB/s 
     |████████████████████████████████| 3.3 MB 59.2 MB/s 
     |████████████████████████████████| 61 kB 505 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import TFBertForQuestionAnswering
from transformers import BertTokenizer 
import tensorflow as tf 
import numpy as np 


In [ ]:
QAModel = TFBertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer_for_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
def bert_answering_machine( question, passage, max_len=512):
  input_ids = tokenizer_for_bert.encode(question, passage, max_length = max_len, truncation=True) #If question+passage longer than 512 truncate.
  sep_index = input_ids.index(102) #Since value [102] defines when we are about to start a new sequence
  len_question = sep_index + 1 #Since it is 0 based and the first sequence is the question, the second is the passage.
  len_answer = len(input_ids)-len_question #Length of answer which is second sequence.
  segment_ids = [0]*len_question + [1]*len_answer #This differentiates the two sequences.
  tokens = tokenizer_for_bert.convert_ids_to_tokens(input_ids) #Converting back to words
  start_token_scores = QAModel(tf.convert_to_tensor([input_ids]), token_type_ids=tf.convert_to_tensor([segment_ids]))[0] #This returns the index of the starting pos of answer
  #This starting position is found in text
  end_token_scores = QAModel(tf.convert_to_tensor([input_ids]),tf.convert_to_tensor([segment_ids]))[1] #Ending index 
  start_token_scores = np.array(start_token_scores).flatten()
  end_token_scores = np.array(start_token_scores).flatten() 
  answer_start_index = np.argmax(start_token_scores)
  end_index = np.argmax(end_token_scores)
  start_token_score = np.round(start_token_scores[answer_start_index], 2)
  end_token_score = np.round(end_token_scores[end_index], 2)
  answer = tokens[answer_start_index]
  for i in range(answer_start_index+1, end_index+1):
    if tokens[i][0:2] == "##":
      answer += tokens[i][2:] #if token starts with "##" eliminate this and then combine with previous part to make full word
    else:
      answer += ' ' + tokens[i] #if token doesnt start with "##" We simply add token with a space seperating their sequence!
  #Now we will do error handling!
  if(answer_start_index ==0) or (start_token_score<0) or (answer ==['[SEP]']) or (end_index < answer_start_index):
    answer = "Sorry!, I could not find answer in passage"
  return (answer_start_index, end_index, start_token_score, end_token_score, answer)
bert_answering_machine("Which city john's friend lives", "My name is John. I live in San Jose, California. Rob is my friend. He lives in Seattle,Washington.")

(30, 30, 6.51, 6.51, 'seattle')

In [ ]:
question = "Whats my favorite vehicle"
passage = "Ken's favorite vehicle is porsche, my name is Ken. I am 16"
bert_answering_machine(question, passage)

(13, 13, 6.82, 6.82, 'porsche')